In [1]:
# Basic imports
import numpy as np
import pandas as pd

# Plotting tools
import bokeh
from bokeh.io import show

import cellseg.plot

bokeh.io.output_notebook()

Loading BokehJS ...

## Figure plotting
The purpose of this code is to generate the figure panels for publication from the data collected by `in_vitro_cell_segmentation.ipynb`

We can start by importing the data that we want to work with:

In [2]:
# Use read_csv to create a dataframe from the data csv 
df_data = pd.read_csv('data/quantification/20220927-data.csv', comment='#')

# Use read_csv to create a dataframe from the look up table
df_meta = pd.read_csv('data/quantification/20220927_data_lookup_table.csv', comment='#')

Then, we can merge the dataframes to connect the metadata with the measured data:

In [3]:
# Use merge the two dataframes, using round, plate, and well to combine them
df = pd.merge(df_data, df_meta, on=["Round", "Plate", "Well"])

We can sort and curate the resulting dataframe: 

In [4]:
#We can sort the values to ensure they're in a logical order
df = df.sort_values(by = ['Round', 'Plate', 'Well'], axis = 0)

# Drop any empty rows and reset the index so that it is in sequence
df = df.dropna().reset_index(drop=True)

We can perform some initial analysis on the data to determine the percent positive area of each well, the scaled area (to enable plotting at a smaller size), and the total brightness per signal area.

In [5]:
# Determine the percent positive and add it to a new column
df['Percent Positive'] = df['Signal Area']*100/df['Bright Field Area']
df['Percent Positive Scaled'] = df['Percent Positive']*0.5
df['Total Brightness per Signal Area'] = df['Total Brightness']/df['Signal Area']

We can then examine the resulting dataframe: 

In [6]:
# Create a index to only include valid data (selected by experimenter)
inds = df['Include'] == True

# Create a reduced data frame of this data
df_reduced = df.loc[inds]

# Look at the dataframe 
df_reduced.head()

,Date,Round,Plate,Well,Count,Cells Quantified,Brightness List,Bright Field Area,Signal Area,Total Brightness,...,90% Confidence Interval,Virus,Receptor,Include,Dose,DMSO (%),Brinzolamide (nM),Percent Positive,Percent Positive Scaled,Total Brightness per Signal Area
0,2022-08-31 18:07:15.091087,1,1,1,293,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.0800825593395253, '2': 0.0833759590792...",166689,14092,3534.819608,...,"[0.064593837535014, 0.4457565667776543]",PHP.eB,Ly6a,True,1.000000e+09,0.0,0.0,8.454067,4.227034,0.250839
1,2022-08-31 18:07:16.537289,1,1,2,408,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.38899619549312264, '2': 0.093681917211...",152698,13466,3199.658824,...,"[0.06732026143790851, 0.43567486253760757]",PHP.eB,Ly6a,True,1.000000e+09,0.0,0.0,8.818714,4.409357,0.237610
2,2022-08-31 18:07:18.101663,1,1,3,501,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.07352941176470587, '2': 0.108650519031...",166536,15691,4430.505882,...,"[0.0784313725490196, 0.49664650907091806]",PHP.eB,Ly6a,True,1.000000e+09,0.0,0.0,9.421987,4.710993,0.282360
3,2022-08-31 18:07:19.502329,1,1,4,351,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.3840607210626185, '2': 0.1131907308377...",164404,12472,3075.960784,...,"[0.06686274509803922, 0.43730278436160797]",PHP.eB,Ly6a,True,5.000000e+08,0.0,0.0,7.586190,3.793095,0.246629
4,2022-08-31 18:07:20.939185,1,1,5,397,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.2784313725490196, '2': 0.1019607843137...",179028,13464,3490.862745,...,"[0.06845938375350141, 0.44557593077716917]",PHP.eB,Ly6a,True,5.000000e+08,0.0,0.0,7.520611,3.760306,0.259274


Now that we have our data, we can use it to generate the figures.

---

In [86]:
def df_simplification(df,
                      receptors = None,
                      viruses = None,
                      rounds = None,
                      doses = None,
                      dmso = None):
    
    receptor_inds = 1
    virus_inds = 1
    round_inds = 1
    doses_inds = 1
    dmso_inds = 1
    
    if receptors != None:
        receptor_inds = df['Receptor'].isin(receptors)
    
    if viruses != None:
        virus_inds = df['Virus'].isin(viruses)
    
    if rounds != None:
        round_inds = df['Round'].isin(rounds)
        
    if doses != None:
        doses_inds = df['Dose'].isin(doses)
        
    if dmso != None:
        dmso_inds = df['DMSO %'].isin(dmso)
    
    inds = np.ones_like(df.index)
    
    for indices in [receptor_inds, virus_inds, round_inds, doses_inds, dmso_inds]:
        inds *= indices
            
    df_fig = df.loc[inds]
    
    df_fig = df_fig.astype({"Dose": str})
    
    return(df_fig)

# Main Figures

## Figure 2

### Part C

First, we want to compare PHP.eB against AAV9 at a range of doses without a receptor.

In [91]:
# We want to make a new dataframe with just the Ly6a and receptor-free conditions.
# First, we can find the indices of these receptors.
receptor_inds = df_reduced['Receptor'].isin(['Ly6a', 'None'])
virus_inds = df_reduced['Virus'].isin(['PHP.eB', 'AAV9'])
round_inds = df_reduced['Round'] == 10

inds = receptor_inds & virus_inds & round_inds

# Make this a new dataframe df_f2c
df_f2c = df_reduced.loc[inds]

# We need to convert the dose values to a string to allow us to plot catagorically
df_f2c = df_f2c.astype({"Dose": str})

# We can determine the mean of each of the values
df_f2c = df_f2c.groupby(['Virus', 'Receptor', 'Dose']).mean(numeric_only = True).reset_index()

In [92]:
# Setting our index to only examine data with Receptor none, we can plot the untransfected AAV9 and PHP.eB transfections
inds = (df_f2c['Receptor'] == 'None')

p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_f2c[inds], 
                                                                               'Virus', 
                                                                               'Dose', 
                                                                               y_values = ['500000000.0', '1000000000.0', '5000000000.0', '10000000000.0', '50000000000.0'], 
                                                                               plot_width = 350, 
                                                                               plot_height = 500)

bokeh.io.show(p)

print("The most positive value is: %0.2f \nThe least positive value is: %0.2f \nThe brightest value is: %0.2f \nThe dimmest value is: %0.2f" % (max_pos, min_pos, max_brightness, min_brightness))

The most positive value is: 74.89 
The least positive value is: 3.14 
The brightest value is: 0.79 
The dimmest value is: 0.39


In [34]:
# Setting our index to only examine data with Receptor none, we can plot the untransfected AAV9 and PHP.eB transfections
inds = (df_f2c['Receptor'] == 'Ly6a')

p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_f2c[inds], 
                                                                               'Virus', 
                                                                               'Dose', 
                                                                               y_values = ['500000000.0', '1000000000.0', '5000000000.0', '10000000000.0', '50000000000.0'], 
                                                                               plot_width = 350, 
                                                                               plot_height = 500)

bokeh.io.show(p)

print("The most positive value is: %0.2f \nThe least positive value is: %0.2f \nThe brightest value is: %0.2f \nThe dimmest value is: %0.2f" % (max_pos, min_pos, max_brightness, min_brightness))

The most positive value is: 39.44 
The least positive value is: 3.99 
The brightest value is: 0.71 
The dimmest value is: 0.40


### Part D

For part D, we want to compare transduction rates across virus and receptor combinations. To do this, we match the dose and compare across rounds. Due to some issues with viral prep, Round 8 was excluded for PHP.C1 and PHP.C3. 

In [10]:
dose_inds = df_reduced['Dose'] == 1e9
round_inds = df_reduced['Round'] <= 7
virus_inds = df_reduced['Virus'].isin(['PHP.eB',
                                       'CAP-B10',
                                       'CAP-B22',
                                       'PHP.N',
                                       'PHP.C1',
                                       'PHP.C2',
                                       'PHP.C3',
                                       'PHP.C4',
                                       'AAV-F'])

receptor_inds = df_reduced['Receptor'].isin(['Tek',
                                             'Slco1c1',
                                             'Slco1a4',
                                             'Slc7a1',
                                             'Slc6a6',
                                             'Slc2a1', 
                                             'Slc22a8',
                                             'Serinc3',
                                             'Prom1',
                                             'Podxl',
                                             'Pecam1', 
                                             'Paqr5', 
                                             'Ocln', 
                                             'Ly6e', 
                                             'Ly6c1', 
                                             'Ly6a', 
                                             'Kitl', 
                                             'Kdr', 
                                             'Itm2b', 
                                             'Igf1r', 
                                             'Ifitm3', 
                                             'Ifitm2', 
                                             'Flt1', 
                                             'Fcgrt', 
                                             'Esam', 
                                             'Eng',
                                             'Clic4', 
                                             'Clic1', 
                                             'Clec2d', 
                                             'Cldn5', 
                                             'Cdh5', 
                                             'Cd34', 
                                             'Car4', 
                                             'Bsg', 
                                             'App', 
                                             'Adgrl4',
                                             'Adgrf5', 
                                             'Acvrl1', 
                                             'Abcg2', 
                                             'Abcb1a'])

inds = receptor_inds & virus_inds & round_inds & dose_inds

df_f2d_1 = df_reduced.loc[inds]

inds = (df['Round'] == 7) & (df['Virus'].isin(['PHP.C1','PHP.C3']))

df_f2d_1 = df_f2d_1.loc[~inds]

# We can determine the mean of each of the values
df_f2d_1 = df_f2d_1.groupby(['Virus', 'Receptor', 'Dose']).mean(numeric_only = True).reset_index()

In [11]:
p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_f2d_1, 
                                                                               'Virus', 
                                                                               'Receptor',
                                                                               x_values = ['PHP.eB',
                                                                                           'CAP-B10',
                                                                                           'CAP-B22',
                                                                                           'PHP.N',
                                                                                           'PHP.C1',
                                                                                           'PHP.C2',
                                                                                           'PHP.C3',
                                                                                           'PHP.C4',
                                                                                           'AAV-F'],
                                                                               y_values = ['Tek',
                                                                                           'Slco1c1',
                                                                                           'Slco1a4',
                                                                                             'Slc7a1',
                                                                                             'Slc6a6',
                                                                                             'Slc2a1', 
                                                                                             'Slc22a8',
                                                                                             'Serinc3',
                                                                                             'Prom1',
                                                                                             'Podxl',
                                                                                             'Pecam1', 
                                                                                             'Paqr5', 
                                                                                             'Ocln', 
                                                                                             'Ly6e', 
                                                                                             'Ly6c1', 
                                                                                             'Ly6a', 
                                                                                             'Kitl', 
                                                                                             'Kdr', 
                                                                                             'Itm2b', 
                                                                                             'Igf1r', 
                                                                                             'Ifitm3', 
                                                                                             'Ifitm2', 
                                                                                             'Flt1', 
                                                                                             'Fcgrt', 
                                                                                             'Esam', 
                                                                                             'Eng',
                                                                                             'Clic4', 
                                                                                             'Clic1', 
                                                                                             'Clec2d', 
                                                                                             'Cldn5', 
                                                                                             'Cdh5', 
                                                                                             'Cd34', 
                                                                                             'Car4', 
                                                                                             'Bsg', 
                                                                                             'App', 
                                                                                             'Adgrl4',
                                                                                             'Adgrf5', 
                                                                                             'Acvrl1', 
                                                                                             'Abcg2', 
                                                                                             'Abcb1a'],
                                                                               plot_width = 600, 
                                                                               plot_height = 1200)

bokeh.io.show(p)

print("The most positive value is: %0.2f \nThe least positive value is: %0.2f \nThe brightest value is: %0.2f \nThe dimmest value is: %0.2f" % (max_pos, min_pos, max_brightness, min_brightness))

The most positive value is: 32.85 
The least positive value is: 0.09 
The brightest value is: 0.55 
The dimmest value is: 0.06


In [12]:
dose_inds = df_reduced['Dose'] == 1e9
round_inds = df_reduced['Round'].isin([7, 8, 9])
virus_inds = df_reduced['Virus'].isin(['9P31',
                                       '9P36'])

inds = dose_inds & round_inds & virus_inds

df_f2d_2 = df_reduced.loc[inds]

# We can determine the mean of each of the values
df_f2d_2 = df_f2d_2.groupby(['Virus', 'Receptor', 'Dose']).mean(numeric_only = True).reset_index()

In [13]:
p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_f2d_2, 
                                                                               'Virus', 
                                                                               'Receptor',
                                                                               x_values = ['9P31',
                                                                                           '9P36'],
                                                                               y_values = ['Tek',
                                                                                           'Slco1c1',
                                                                                           'Slco1a4',
                                                                                             'Slc7a1',
                                                                                             'Slc6a6',
                                                                                             'Slc2a1', 
                                                                                             'Slc22a8',
                                                                                             'Serinc3',
                                                                                             'Prom1',
                                                                                             'Podxl',
                                                                                             'Pecam1', 
                                                                                             'Paqr5', 
                                                                                             'Ocln', 
                                                                                             'Ly6e', 
                                                                                             'Ly6c1', 
                                                                                             'Ly6a', 
                                                                                             'Kitl', 
                                                                                             'Kdr', 
                                                                                             'Itm2b', 
                                                                                             'Igf1r', 
                                                                                             'Ifitm3', 
                                                                                             'Ifitm2', 
                                                                                             'Flt1', 
                                                                                             'Fcgrt', 
                                                                                             'Esam', 
                                                                                             'Eng',
                                                                                             'Clic4', 
                                                                                             'Clic1', 
                                                                                             'Clec2d', 
                                                                                             'Cldn5', 
                                                                                             'Cdh5', 
                                                                                             'Cd34', 
                                                                                             'Car4', 
                                                                                             'Bsg', 
                                                                                             'App', 
                                                                                             'Adgrl4',
                                                                                             'Adgrf5', 
                                                                                             'Acvrl1', 
                                                                                             'Abcg2', 
                                                                                             'Abcb1a'],
                                                                               plot_width = 300, 
                                                                               plot_height = 1200)

bokeh.io.show(p)

print("The most positive value is: %0.2f \nThe least positive value is: %0.2f \nThe brightest value is: %0.2f \nThe dimmest value is: %0.2f" % (max_pos, min_pos, max_brightness, min_brightness))

The most positive value is: 55.28 
The least positive value is: 3.03 
The brightest value is: 0.61 
The dimmest value is: 0.11


### Part G

In [14]:
round_inds = df_reduced['Round'] == 13
dose_ids = df_reduced['Dose'] == 1e9

inds = round_inds & dose_inds

# Make this a new dataframe df_edf5
df_f2g_1 = df_reduced.loc[inds]

# We can determine the mean of each of the values
df_f2g_1 = df_f2g_1.groupby(['Virus', 'Receptor', 'Dose']).mean(numeric_only = True).reset_index()

In [15]:
p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_f2g_1, 
                                                                               'Virus',
                                                                               'Receptor',
                                                                               x_values = ['PHP.eB',
                                                                                           'CAP-B10',
                                                                                           'CAP-B22',
                                                                                           '9P08',
                                                                                           '9P16',
                                                                                           'PHP.N',
                                                                                           'PHP.C1',
                                                                                           'PHP.C2',
                                                                                           'PHP.C3',
                                                                                           'PHP.C4',
                                                                                           'AAV-F',
                                                                                           '9P13',
                                                                                           '9P33',
                                                                                           '9P39'],
                                                                               y_values = ['CD59',
                                                                                           'Ly6S',
                                                                                           'Lynx1',
                                                                                           'Lypd5',
                                                                                           'Ly6h',
                                                                                           'Ly6e', 
                                                                                           'Ly6c1',
                                                                                           'Ly6a'],
                                                                               plot_width = 700, 
                                                                               plot_height = 400)

bokeh.io.show(p)

print("The most positive value is: %0.2f \nThe least positive value is: %0.2f \nThe brightest value is: %0.2f \nThe dimmest value is: %0.2f" % (max_pos, min_pos, max_brightness, min_brightness))

The most positive value is: 50.54 
The least positive value is: 2.54 
The brightest value is: 0.75 
The dimmest value is: 0.13


In [16]:
round_inds = df_reduced['Round'] == 10
dose_inds = df_reduced['Dose'] == 1e9

inds = round_inds & dose_inds

df_f2g_2 = df_reduced.loc[inds]

# We can determine the mean of each of the values
df_f2g_2 = df_f2g_2.groupby(['Virus', 'Receptor', 'Dose']).mean(numeric_only = True).reset_index()

In [17]:
p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_f2g_2, 
                                                                               'Virus',
                                                                               'Receptor',
                                                                               x_values = ['9P31',
                                                                                          '9P36'],
                                                                               y_values = ['Car15',
                                                                                          'Car14',
                                                                                          'Car12',
                                                                                          'Car9',
                                                                                          'Car4'],
                                                                               plot_width = 250, 
                                                                               plot_height = 400)

bokeh.io.show(p)

print("The most positive value is: %0.2f \nThe least positive value is: %0.2f \nThe brightest value is: %0.2f \nThe dimmest value is: %0.2f" % (max_pos, min_pos, max_brightness, min_brightness))

The most positive value is: 56.51 
The least positive value is: 1.86 
The brightest value is: 0.79 
The dimmest value is: 0.34


## Figure 4

### Part C

In [18]:
round_inds = df_reduced['Round'] == 11
dose_inds = df_reduced['Dose'].isin([1e7, 1e8, 1e9])

inds = round_inds & dose_inds

df_f4c = df_reduced.loc[inds]

df_f4c = df_f4c.astype({"Dose": str})

df_f4c = df_f4c.groupby(['Virus', 'Receptor', 'Dose']).mean(numeric_only = True).reset_index()

In [19]:
for virus in ['PHP.eB', 'PHP.C2', 'eC 19']:
    inds = (df_f4c['Virus'] == virus)

    p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_f4c.loc[inds],
                                                                                   'Receptor',
                                                                                   'Dose',
                                                                                   x_values = ['None',
                                                                                               'Ly6c1'],
                                                                                   y_values = ['10000000.0',
                                                                                               '100000000.0',
                                                                                               '1000000000.0'],
                                                                               plot_width = 300, 
                                                                               plot_height = 400)
    bokeh.io.show(p)
    
    print("The most positive value is: %0.2f \nThe least positive value is: %0.2f \nThe brightest value is: %0.2f \nThe dimmest value is: %0.2f" % (max_pos, min_pos, max_brightness, min_brightness))

The most positive value is: 11.10 
The least positive value is: 2.45 
The brightest value is: 0.29 
The dimmest value is: 0.14


The most positive value is: 39.05 
The least positive value is: 3.07 
The brightest value is: 0.65 
The dimmest value is: 0.15


The most positive value is: 58.98 
The least positive value is: 3.08 
The brightest value is: 0.74 
The dimmest value is: 0.16


## Fig 5

### Part D

In [20]:
virus_inds = df_reduced['Virus'] == 'PHP.eB'
round_inds = df_reduced['Round'] == 12
dose_inds = df_reduced['Dose'] != 1e10

inds = virus_inds & round_inds & dose_inds

df_f5d = df_reduced.loc[inds]

df_f5d = df_f5d.astype({"Dose": str})

df_f5d = df_f5d.groupby(['Virus', 'Receptor', 'Dose']).mean(numeric_only = True).reset_index()

In [21]:
p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_f5d, 
                                                                               'Dose',
                                                                               'Receptor',
                                                                               x_values = ['1000000000.0',
                                                                                           '100000000.0',
                                                                                           '10000000.0'],
                                                                               y_values = ['Ly6a A58R',
                                                                                           'Ly6a'],
                                                                               plot_width = 300, 
                                                                               plot_height = 200)

bokeh.io.show(p)

print("The most positive value is: %0.2f \nThe least positive value is: %0.2f \nThe brightest value is: %0.2f \nThe dimmest value is: %0.2f" % (max_pos, min_pos, max_brightness, min_brightness))

The most positive value is: 28.70 
The least positive value is: 2.94 
The brightest value is: 0.61 
The dimmest value is: 0.16


## Figure 6

### Part B

In [22]:
dmso_inds = df_reduced['DMSO (%)'] == 0.3 
dose_inds = df_reduced['Dose'] == 1e9

inds = dmso_inds & dose_inds

df_f6b = df_reduced.loc[inds]

# We need to convert the dose values to a string to allow us to plot catagorically
df_f6b = df_f6b.astype({"Brinzolamide (nM)": str})
df_f6b = df_f6b.astype({"Dose": str})

# We can determine the mean of each of the values
df_f6b = df_f6b.groupby(['Virus', 'Receptor', 'Dose', 'Brinzolamide (nM)']).mean(numeric_only = True).reset_index()

In [23]:
inds = (df_f6b['Receptor'] == 'msCar4')

p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_f6b[inds], 
                                                                               'Brinzolamide (nM)',
                                                                               'Virus',
                                                                               x_values = ['0.0',
                                                                                           '1.0',
                                                                                           '100.0',
                                                                                           '300.0'],
                                                                               y_values = ['9P36',
                                                                                           '9P31'],
                                                                               plot_width = 400, 
                                                                               plot_height = 200)

bokeh.io.show(p)

print("The most positive value is: %0.2f \nThe least positive value is: %0.2f \nThe brightest value is: %0.2f \nThe dimmest value is: %0.2f" % (max_pos, min_pos, max_brightness, min_brightness))

The most positive value is: 63.26 
The least positive value is: 3.37 
The brightest value is: 0.75 
The dimmest value is: 0.12


In [24]:
inds = (df_f6b['Receptor'] == 'Ly6a')

p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_f6b[inds], 
                                                                               'Brinzolamide (nM)',
                                                                               'Virus',
                                                                               x_values = ['0.0',
                                                                                           '1.0',
                                                                                           '100.0',
                                                                                           '300.0'],
                                                                               y_values = ['PHP.eB'],
                                                                               plot_width = 400, 
                                                                               plot_height = 100)

bokeh.io.show(p)

print("The most positive value is: %0.2f \nThe least positive value is: %0.2f \nThe brightest value is: %0.2f \nThe dimmest value is: %0.2f" % (max_pos, min_pos, max_brightness, min_brightness))

The most positive value is: 22.76 
The least positive value is: 5.25 
The brightest value is: 0.51 
The dimmest value is: 0.18


### Part D

In [25]:
round_inds = df_reduced['Round'] == 16
dose_inds = df_reduced['Dose'] == 1e9

inds = round_inds & dose_inds

df_f6d_1 = df_reduced[inds]

df_f6d_1 = df_f6d_1.groupby(['Virus', 'Receptor', 'Dose']).mean(numeric_only = True).reset_index()

p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_f6d_1,
                                                                               'Receptor',
                                                                               'Virus',
                                                                               x_values = ['msCar4',
                                                                                           'rhCar4',
                                                                                           'huCar4'],
                                                                               y_values = ['9P36', 
                                                                                           '9P31'],
                                                                               plot_width = 400, 
                                                                               plot_height = 250)

bokeh.io.show(p)

print("The most positive value is: %0.2f \nThe least positive value is: %0.2f \nThe brightest value is: %0.2f \nThe dimmest value is: %0.2f" % (max_pos, min_pos, max_brightness, min_brightness))

The most positive value is: 57.15 
The least positive value is: 6.22 
The brightest value is: 0.80 
The dimmest value is: 0.21


In [26]:
round_inds = df_reduced['Round'] == 18
dose_inds = df_reduced['Dose'] == 1e9

inds = round_inds & dose_inds

df_f6d_2 = df_reduced[inds]

df_f6d_2 = df_f6d_2.groupby(['Virus', 'Receptor', 'Dose']).mean(numeric_only = True).reset_index()

p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_f6d_2,
                                                                               'Receptor',
                                                                               'Virus',
                                                                               x_values = ['None',
                                                                                           'msCar4',
                                                                                           'TM 122',
                                                                                           'TM 123'],
                                                                               y_values = ['9P36', 
                                                                                           '9P31'],
                                                                               plot_width = 400, 
                                                                               plot_height = 250)

bokeh.io.show(p)

print("The most positive value is: %0.2f \nThe least positive value is: %0.2f \nThe brightest value is: %0.2f \nThe dimmest value is: %0.2f" % (max_pos, min_pos, max_brightness, min_brightness))

The most positive value is: 64.69 
The least positive value is: 3.26 
The brightest value is: 0.73 
The dimmest value is: 0.15


## Supplemental Figures

### Figure S2c

In [27]:
round_inds = df_reduced['Round'] == 17
dose_inds = df_reduced['Dose'] == 1e9
virus_inds = df_reduced['Virus'].isin(['PHP.eB', 'CAP-B10', 'CAP-B22'])

inds = round_inds & dose_inds * virus_inds
                                      
df_sf2a = df_reduced[inds]
                                      
df_sf2a = df_sf2a.groupby(['Virus', 'Receptor', 'Dose']).mean(numeric_only = True).reset_index()

In [28]:
p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_sf2a,
                                                                               'Receptor',
                                                                               'Virus',
                                                                               x_values = ['None',
                                                                                           'Ly6a',
                                                                                           'Marmoset Ly6E',
                                                                                           'Marmoset LyPD5',
                                                                                           'Marmoset Lynx1',
                                                                                           'Marmoset CD59'],
                                                                               y_values = ['CAP-B22',
                                                                                           'CAP-B10',
                                                                                           'PHP.eB'],
                                                                               plot_width = 400, 
                                                                               plot_height = 300)

bokeh.io.show(p)

In [29]:
round_inds = df_reduced['Round'] == 17
dose_inds = df_reduced['Dose'] == 1e9
virus_inds = df_reduced['Virus'] == 'PHP.C2'
receptor_inds = df_reduced['Receptor'].isin(['None', 'Ly6c1', 'Ly6c1 E31R', 'Ly6c1 Q56A', 'Ly6c1 S94E', 'Ly6c1 triple mutant'])

inds = round_inds & dose_inds & virus_inds & receptor_inds

df_sf5e = df_reduced.loc[inds]

# We can determine the mean of each of the values
df_sf5e = df_sf5e.groupby(['Virus', 'Receptor', 'Dose']).mean(numeric_only = True).reset_index()

In [30]:
p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_sf5e,
                                                                               'Receptor',
                                                                               'Virus',
                                                                               x_values = ['None',
                                                                                           'Ly6c1', 
                                                                                           'Ly6c1 E31R', 
                                                                                           'Ly6c1 Q56A',
                                                                                           'Ly6c1 S94E'],
                                                                               y_values = ['PHP.C2'],
                                                                               plot_width = 500, 
                                                                               plot_height = 150)

bokeh.io.show(p)